# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Kelly\Dropbox\Insper Aulas\DataScience (1)\2020.1\aulas\aula07 projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [ ]:
filename = 'COLOQUE NOME DO SEU ARQUIVO AQUI.xlsx'

In [5]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento
0,@isa__ferrari te amo amor ❤️❤️
1,"@ferrari_carol miga, eu to torcendo tanto daqu..."
2,"@madu_ferrari_ mds, parece a gente versão mais..."
3,ela: um dia você vai usar minha blusa da ferra...
4,dps n sabe pq tem dengue https://t.co/5e4dvpdfvl


In [9]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste
0,@ferrari_carol já tem mais de 7 anos que tenho...
1,"rt @nizzolaarthur: mt cuidado galera, tem mt g..."
2,@isa__ferrari to indo mor
3,@jsenna12 @ivo_pereira03 classificação este an...
4,"já dizia o poeta: ""improvisar berrío na latera..."


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**